Uploading data to GCP from VM:
-----------------------------------------------------------------------
- authentication is neccessary: gcloud auth login
- navigate to data directory you want upload
- run: gsutil -m cp -r pq/ gs://dtc_data_lake_de-zoomcamp-nytaxi_w5/pq

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
pyspark.__version__

'3.4.1'

In [3]:
# Stop active SparkSession, if any
#spark = SparkSession.builder.getOrCreate()
#spark.stop()

24/03/01 14:06:06 WARN Utils: Your hostname, MacBook-Pro-von-Sebastian.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/03/01 14:06:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 14:06:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
credentials_location = 'dataengineering-secret.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location) #\
    #.set("spark.ui.port", "4050") \
    #.set("spark.ui.enabled", "false")


In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/01 14:08:55 WARN Utils: Your hostname, MacBook-Pro-von-Sebastian.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
24/03/01 14:08:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/01 14:08:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df_green = spark.read.parquet('gs://dtc_data_lake_de-zoomcamp-nytaxi_w5/pq/green/*/*')

In [6]:
df_green.count()

2304517